<a href="https://colab.research.google.com/github/prteek/data-science/blob/master/DecisionTrees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Trees
*A tree is an incomprehensible mystery. -Jim Woodring*

In [0]:
# This cell is not required to be executed (i.e. ignore any error) if Notebook is run locally or in Binder
# Authorise and mount google drive to access code and data files
import os
from google.colab import drive
drive.mount('/content/drive')
project_folder = '/content/drive/My Drive/git_repos/data-science'
os.chdir(project_folder)

In [0]:
%%capture
# To supress the output when calling Statistics file
%run ./LogisticRegression.ipynb

In [0]:
# Entropy
def entropy(class_probabilities):
    """given a list of class probabilities, compute the entropy"""
    return sum(-p*math.log(p,2)
              for p in class_probabilities
              if p) # ignore zero probabilities

def class_probabilities(labels):
    total_count = len(labels)
    return [count/total_count
           for count in Counter(labels).values()]

def data_entropy(labeled_data):
    """labeled_data is in the form of (input, label)"""
    labels = [label for _, label in labeled_data]
    probabilities = class_probabilities(labels)
    return entropy(probabilities)

# Entropy of a partition
def partition_entropy(subsets):
    """find the entropy from this partition of the data into subsets
    subsets is a list of lists of labeled data"""
    total_count = sum(len(subset) for subset in subsets)
    return sum(data_entropy(subset)*len(subset)/total_count
              for subset in  subsets)


### Creating a Decision tree

In [0]:
inputs = [
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'no'},   False),
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'yes'},  False),
        ({'level':'Mid','lang':'Python','tweets':'no','phd':'no'},     True),
        ({'level':'Junior','lang':'Python','tweets':'no','phd':'no'},  True),
        ({'level':'Junior','lang':'R','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'R','tweets':'yes','phd':'yes'},    False),
        ({'level':'Mid','lang':'R','tweets':'yes','phd':'yes'},        True),
        ({'level':'Senior','lang':'Python','tweets':'no','phd':'no'}, False),
        ({'level':'Senior','lang':'R','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'Python','tweets':'yes','phd':'no'}, True),
        ({'level':'Senior','lang':'Python','tweets':'yes','phd':'yes'},True),
        ({'level':'Mid','lang':'Python','tweets':'no','phd':'yes'},    True),
        ({'level':'Mid','lang':'Java','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'Python','tweets':'no','phd':'yes'},False)
    ]


# The data set has both TRUE and FALSE labels, and we have 4 attributes we can split on.
# So the first step will be to find the partition with least entropy.
# The function below does the partitioning by given attribute

def partition_by(inputs, attribute):
    """each input is a pair (attribute_dict, label)
    returns a dict : attribute_value -> inputs"""
    groups = defaultdict(list)
    for input_i in inputs:
        key = input_i[0][attribute]  # get the value of the specified attribute
        groups[key].append(input_i)  # then add this input to the correct list
    return groups

# The function below uses it to compute entropy
def partition_entropy_by(inputs, attribute, partitions_output=False):
    """computes the entropy corresponding to the given partition"""
    partitions = partition_by(inputs, attribute)
    if partitions_output:
        return partitions, partition_entropy(partitions.values())
    else:
        return partition_entropy(partitions.values())

# Then we can just find the minimum entropy partition for the whole dataset
for key in ['level', 'lang', 'tweets', 'phd']:
    parts, parts_entropy = partition_entropy_by(inputs, key, True)
    print(key,':', round(parts_entropy,2), 
           [len(part) for part in parts.values()]) # in each attribute how many entries in each values 

# The lowest entropy comes from splitting on "level", so we need ti make a subtree for each possible level. 
# Doing this shows that every "Mid" candidate is labeled TRUE, which means that the "Mid" subtree is simply
# a leaf node predicting TRUE.
print("----------------")

# For "Senior" candidates we have a mix of TRUE and FALSE, so we need to split again.

senior_inputs = [(i_input, label) for i_input, label in inputs if i_input["level"]== "Senior"]

for key in ['lang', 'tweets', 'phd']:
    parts, parts_entropy = partition_entropy_by(senior_inputs, key, True)
    print(key,':', round(parts_entropy,2), 
           [len(part) for part in parts.values()]) # in each attribute how many entries in each values 

# This shows that the next split should be on "Tweets", which results in a zero-entropy partition.
# For the "Senior" level candidates, "yes" tweets always result in TRUE, while "no" tweets result in FALSE.
print("----------------")

# Finally we do the same thing for the "Junior" candidates, we end up splitting on Phd, after which...
# we find "no" Phd always results in TRUE and Phd always results in FALSE

junior_inputs = [(i_input, label) for i_input, label in inputs if i_input["level"]== "Junior"]

for key in ['lang', 'tweets', 'phd']:
    parts, parts_entropy = partition_entropy_by(junior_inputs, key, True)
    print(key,':', round(parts_entropy,2), 
           [len(part) for part in parts.values()]) # in each attribute how many entries in each values 

### Putting it all together
We define a tree to be one of the following: <br/>
**True** <br/>
**False** <br/>
**A tuple (attribute, subtree_dict)** <br/>

*True* or *False* represents a "leaf node" for any input, *tuple* represents a "decision node" that, for any input, finds its "attribute" value, and classifies the input using the corresponding sub-tree. <br/>
With this representation, our hiring tree would look like:

In [0]:
tree = ('level', {'Junior': ('phd', {'no':True, 'yes':False}),
                  'Mid':True,
                  'Senior': ('tweets', {'no':False, 'yes':True})}
       )

# When we encounter an unexpected (or missing) attribute value, 
# we add a "None" key that just predicts the most common label. (Doesn't work if "None" is actually a value in data).

# To classify an input using a given tree
def classify(tree, input_i):
    """classify the input using the given decision tree"""
    
    # if this is a leaf node, return its value
    if tree in [True, False]:
        return tree
    
    # otherwise this tree consists of an attribute to split on,
    # and a dictionary whose keys are values of that attribute
    # and whose values are subtrees to consider next
    
    attribute, subtree_dict = tree
    subtree_key             = input_i.get(attribute) # None if input is missing attribute
    
    if subtree_key not in subtree_dict: # if no subtree for key,
        subtree_key = None              # we'll use None subtree
        
    subtree = subtree_dict[subtree_key]  # choose appropriate subtree
    return classify(subtree, input_i)    # and use it to classify the input

# Building a tree
def build_tree_id3(inputs, split_candidates=None):
    
    # If this is our first pass, all the keys of the first input are split candidates
    if split_candidates is None:
    # inputs is a "list of tuples" in which first entry of pair is a dictionary of input data
        split_candidates = inputs[0][0].keys() 
    
    # count Trues and Falses in the inputs
    num_inputs  = len(inputs)
    num_trues   = len([label for item, label in inputs if label])
    num_falses  = num_inputs - num_trues
    
    if num_trues == 0: return False # no Trues ? return a "False" leaf
    if num_falses == 0: return True # no Falses ? return a "True" leaf
    
    if not split_candidates:            # if no split_candidates left
        return num_trues >= num_falses  # return the majority leaf
    
    # otherwise, split on the best attribute
    best_attribute = min(split_candidates,
                        key=partial(partition_entropy_by, inputs))
    
    partitions = partition_by(inputs, best_attribute)
    new_candidates = [a for a in split_candidates
                     if a != best_attribute]
    
    # recursively build subtrees
    subtrees = {attribute_value: build_tree_id3(subset, new_candidates)
               for attribute_value, subset in partitions.items()}
    
    subtrees[None] = num_trues > num_falses  # default case
    return (best_attribute, subtrees)
        

tree = build_tree_id3(inputs)

print("Junior/Java/tweets/no phd: ", 
      classify(tree, {"level":"Junior", "lang":"Java", "tweets":"yes", "phd":"no"}))
print("Junior/Java/tweets/phd: ",
      classify(tree, {"level":"Junior", "lang":"Java", "tweets":"yes", "phd":"yes"}), "\n----")

# Handling missing or unexpected values of Attributes
print("Intern: ", classify(tree, {"level":"Intern"}))
print("Senior: ",classify(tree, {"level":"Senior"}))

### Random Forests

In [0]:
def forest_classify(trees, input):
    votes = [classify(tree, input) for tree in trees]
    vote_counts = Counter(votes)
    return vote_counts.most_common(1)[0][0]